# Imports and Setups

In [36]:
# Import necessary libraries
import os
import csv
import re
import logging
import multiqc
import numpy as np
from collections import Counter, OrderedDict
from multiqc import BaseMultiqcModule
from multiqc.plots import linegraph, bargraph, table, scatter
from IPython.display import IFrame, display

# Define paths for the input and output
tsv_folder_path = "/path/to/your/tsv/files/general"
report_path = "/path/to/report"
plots_path = os.path.join(report_path, 'plots')
os.makedirs(plots_path, exist_ok=True)

# Configure logging
log = logging.getLogger('multiqc')
log.setLevel(logging.DEBUG) 

if not log.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    log.addHandler(handler)

# Remove existing MultiQC modules

In [37]:
# Function to remove any existing modules with the same name
def remove_existing_module(module_name):
    existing_modules = [m for m in multiqc.report.modules if m.name == module_name]
    for m in existing_modules:
        multiqc.report.modules.remove(m)

# Remove old module before adding the new one
remove_existing_module("General Statistics MHCquant")

# Initialize MultiQC Module

In [38]:
# Initialize and configure the MultiQC module with a custom name
module = BaseMultiqcModule(name="General Statistics MHCquant", anchor="custom_data")

# Load and Process Sequences for Peptide Length Distribution

In [39]:
# Function to load sequences from TSV files
def load_sequences(folder_path):
    sequence_dict = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                if 'sequence' not in headers:
                    continue  # Skip files without 'sequence' column
                seq_index = headers.index('sequence')
                sequences = [re.sub(r'\(.*?\)', '', row[seq_index]) for row in rows]
                sequence_dict[file_name] = sequences
    return sequence_dict

# Load and process data for peptide length distribution
sample_sequence_dict = load_sequences(tsv_folder_path)
data = {
    sample: dict(Counter(list(map(len, seqs)))) for sample, seqs in sample_sequence_dict.items()
}

# Normalize data and multiply frequency values by 100 for percentage representation
data = {
    sample: {length: (count / sum(seq_len_count.values())) * 100 for length, count in seq_len_count.items()}
    for sample, seq_len_count in data.items()
}

# Configuration for peptide length distribution plot
pconfig = {
    'id': 'peptide_length_distribution',
    'title': 'Peptide Length Distribution',
    'xlab': 'Peptide Length',
    'ylab': 'Frequency [%]',  # y-axis label reflects percentage values
}


# Add section with the modified data for plotting
module.add_section(
    plot=linegraph.plot(data, pconfig=pconfig),
    name="Peptide length distribution",
    anchor="my_metrics_section",
)



# Load and Process m/z Values for Distribution Plot

In [40]:
# Function to load m/z values from TSV files
def load_mz_values(folder_path):
    mz_dict = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                if 'mz' not in headers:
                    log.warning(f"WARNING: 'mz' column not found in {file_name}")
                    continue  # Skip files without 'mz' column
                mz_index = headers.index('mz')
                mz_values = [float(row[mz_index]) for row in rows if row[mz_index]]
                mz_dict[file_name] = mz_values
    return mz_dict

# Load and bin m/z values for bar plot
sample_mz_dict = load_mz_values(tsv_folder_path)

def bin_mz_values(mz_values, bin_size=5):
    min_mz = int(min(mz_values))
    max_mz = int(max(mz_values))
    bins = list(range(min_mz, max_mz + bin_size, bin_size))
    bin_counts = {f"{b}-{b + bin_size}": 0 for b in bins[:-1]}
    for mz in mz_values:
        for b in bins[:-1]:
            if b <= mz < b + bin_size:
                bin_counts[f"{b}-{b + bin_size}"] += 1
                break
    return bin_counts

mz_bin_counts = {sample: bin_mz_values(mz_values) for sample, mz_values in sample_mz_dict.items()}
barplot_data = OrderedDict()
for sample, bin_counts in mz_bin_counts.items():
    for bin_range, count in bin_counts.items():
        if bin_range not in barplot_data:
            barplot_data[bin_range] = {}
        barplot_data[bin_range][sample] = count

# Configuration for m/z distribution bar plot
mz_pconfig = {
    'id': 'mz_distribution',
    'title': 'm/z Distribution',
    'xlab': 'm/z',
    'ylab': 'Frequency',
    'stacked': False,
}
module.add_section(
    plot=bargraph.plot(barplot_data, pconfig=mz_pconfig),
    name="Combined m/z Distribution",
    anchor="mz_distribution_combined_section",
)

# Calculate and Display General Statistics

In [41]:
# Function to calculate statistics for General Stats Table
def calculate_stats(folder_path):
    num_peptides = {}
    num_modified_peptides = {}
    num_proteins = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.DictReader(f, delimiter='\t')
                sequences = set()
                modified_peptides = set()
                proteins = set()
                for row in reader:
                    sequence = row.get('sequence', '')
                    if sequence:
                        sequences.add(sequence)
                        if '(' in sequence and ')' in sequence:
                            modified_peptides.add(sequence)
                    for col in ['accessions', 'protein_references']:
                        if col in row and row[col]:
                            proteins.update(row[col].split(','))
                num_peptides[file_name] = len(sequences)
                num_modified_peptides[file_name] = len(modified_peptides)
                num_proteins[file_name] = len(proteins)
    return num_peptides, num_modified_peptides, num_proteins

# Perform calculations for General Stats Table
num_peptides, num_modified_peptides, num_proteins = calculate_stats(tsv_folder_path)
general_stats_data = {
    file_name: {
        'Number of Peptides': num_peptides[file_name],
        'Number of Modified Peptides': num_modified_peptides[file_name],
        'Number of Protein Groups': num_proteins[file_name]
    }
    for file_name in num_peptides.keys()
}
headers = {
    'Number of Peptides': {'title': 'Number of Peptides', 'description': 'Total number of peptides'},
    'Number of Modified Peptides': {'title': 'Number of Modified Peptides', 'description': 'Peptides with modifications'},
    'Number of Protein Groups': {'title': 'Number of Protein Groups', 'description': 'Total number of protein groups'}
}
table_config = {
    'id': 'custom_general_stats',
    'title': 'Custom Statistics Table',
    'namespace': 'custom_data'
}
plot = table.plot(data=general_stats_data, headers=headers, pconfig=table_config)
module.add_section(
    plot=plot,
    name="Custom General Statistics",
    anchor="custom_general_stats"
)

# Load and Plot Score Data

In [42]:
# Function to load scores from TSV files
def load_score(folder_path):
    score_dict = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                if 'score' not in headers:
                    log.warning(f"WARNING: 'score' column not found in {file_name}")
                    continue
                score_index = headers.index('score')
                score_values = [float(row[score_index]) for row in rows if row[score_index]]
                score_dict[file_name] = score_values
    return score_dict

# Load score data
sample_score_dict = load_score(tsv_folder_path)

# Function to compute the optimal number of bins using Freedman-Diaconis rule
def optimal_bins(data):
    q25, q75 = np.percentile(data, [25, 75])  # First and third quartiles
    bin_width = 2 * (q75 - q25) / len(data) ** (1/3)  # Freedman-Diaconis formula
    bins = int((max(data) - min(data)) / bin_width)
    return bins

# Create bin counts for each sample's scores using optimal number of bins
def bin_score_values_optimal(score_values, bin_size):
    bins = np.linspace(min(score_values), max(score_values), bin_size + 1)
    bin_counts = OrderedDict((f"{round(bins[i], 5)}-{round(bins[i+1], 5)}", 0) for i in range(len(bins) - 1))
    for score in score_values:
        for i in range(len(bins) - 1):
            if bins[i] <= score < bins[i + 1]:
                bin_counts[f"{round(bins[i], 5)}-{round(bins[i + 1], 5)}"] += 1
                break
    return bin_counts

# Calculate optimal bins and create bin counts for each sample
score_bin_counts = {}
for sample, score_values in sample_score_dict.items():
    bin_size = optimal_bins(score_values)  # Calculate the optimal number of bins
    bin_counts = bin_score_values_optimal(score_values, bin_size)
    score_bin_counts[sample] = bin_counts

# Prepare data for the bar plot
score_barplot_data = OrderedDict()
for bin_range in score_bin_counts[next(iter(score_bin_counts))].keys():  # Use the bins from the first sample as reference
    score_barplot_data[bin_range] = {sample: score_bin_counts[sample].get(bin_range, 0) for sample in score_bin_counts}

# Configuration for the score distribution bar plot
score_pconfig = {
    'id': 'score_distribution_combined',
    'title': 'Q-Value Distribution for All Samples',
    'xlab': 'Score Range',
    'ylab': 'Frequency',
    'stacked': True , # Set to True to display stacked bar plot with different colors for each file
}

# Add the score distribution bar plot section
if score_barplot_data:  # Ensure there is data to plot
    module.add_section(
        plot=bargraph.plot(score_barplot_data, pconfig=score_pconfig),
        name="Q-Value Distribution of all Samples",
        anchor="score_distribution_combined_section",
    )
else:
    print("No data available for score distribution plot.")


# Load and Plot Retention Time Data

In [43]:
# Function to load predicted and observed retention times from TSV files
def load_rt_and_predicted_rt(folder_path):
    scatter_data = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                # Check if required columns exist
                if 'predicted_retention_time_best' not in headers or 'observed_retention_time_best' not in headers:
                    log.warning(f"WARNING: 'predicted_retention_time_best' or 'observed_retention_time_best' column not found in {file_name}")
                    continue
                # Get indices of the required columns
                predicted_rt_index = headers.index('predicted_retention_time_best')
                observed_rt_index = headers.index('observed_retention_time_best')
                points = []
                for row in rows:
                    try:
                      predicted_rt_value = float(row[predicted_rt_index])
                      observed_rt_value = float(row[observed_rt_index])
                      points.append({"x": observed_rt_value, "y": predicted_rt_value})
                    except ValueError:
                        continue
                if points:
                    scatter_data[file_name] = points
    log.warning(f"Scatter plot data: {scatter_data}")  # Debugging: Print scatter plot data
    return scatter_data
#Load the updated retention time data
scatter_plot_data = load_rt_and_predicted_rt(tsv_folder_path)#
#Create a scatter plot for each file separately
if scatter_plot_data:
    for file_name, data_points in scatter_plot_data.items():
        scatter_pconfig = {
            'id': f'scatter_{file_name}',
            'title': f'Scatter Plot: Predicted vs Observed Retention Time ({file_name})',
            'xlab': 'Observed Retention Time (RT)',
            'ylab': 'Predicted Retention Time',
            'showlegend': True  # Show legend for each plot
        }
        # Create the scatter plot for the current file
        module.add_section(
            plot=scatter.plot({file_name: data_points}, pconfig=scatter_pconfig),  # Provide data for the current file
            name=f"Predicted vs Observed Retention Time Correlation ({file_name})",
            anchor=f"rt_vs_predicted_rt_correlation_section_{file_name}",
       )
else:
    print("No data available for scatter plots.")


2024-09-20 12:26:56,839 - multiqc - WARNING - Scatter plot data: {'DN02_Brain_1_mqc.tsv': [{'x': 704.3, 'y': 839.7693790485656}, {'x': 757.7, 'y': 813.6430502365524}, {'x': 759.1, 'y': 813.8130268818855}, {'x': 777.9, 'y': 815.6028904417429}, {'x': 783.6, 'y': 841.696565811316}, {'x': 788.0, 'y': 870.2487094688585}, {'x': 816.8, 'y': 890.7678106407207}, {'x': 829.8, 'y': 839.8464308545832}, {'x': 854.3, 'y': 940.487154482095}, {'x': 912.0, 'y': 1022.3689683460317}, {'x': 920.7, 'y': 1027.5489473733066}, {'x': 975.5, 'y': 1071.7050045919475}, {'x': 1002.9, 'y': 1092.7402815806481}, {'x': 1017.3, 'y': 1084.0649069237477}, {'x': 1077.9, 'y': 1085.5678205632}, {'x': 1082.3, 'y': 1133.139267069316}, {'x': 1101.0, 'y': 1122.0248945390797}, {'x': 1103.9, 'y': 1140.537859482252}, {'x': 1111.1, 'y': 1147.5375563770115}, {'x': 1124.1, 'y': 1192.0513265132543}, {'x': 1135.7, 'y': 1179.1008096167077}, {'x': 1148.6, 'y': 1174.04377501859}, {'x': 1151.5, 'y': 1159.6603652485687}, {'x': 1160.2, 'y': 

# Load and Plot Peptide Counts over RT

In [44]:
# Function to load data and calculate peptide counts over retention time
def load_peptide_counts_by_rt(folder_path):
    rt_data = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                if 'observed_retention_time_best' not in headers:
                    log.warning(f"WARNING: 'observed_retention_time_best' column not found in {file_name}")
                    continue
                rt_index = headers.index('observed_retention_time_best')
                rt_values = [float(row[rt_index]) for row in rows if row[rt_index]]
                
                # Define bins for retention time
                rt_bins = [round(min(rt_values) + i * 2, 2) for i in range(int((max(rt_values) - min(rt_values)) / 2) + 1)]
                rt_counts = OrderedDict((bin_start, 0) for bin_start in rt_bins)

                # Count the number of peptides in each bin
                for rt in rt_values:
                    for bin_start in rt_bins:
                        if bin_start <= rt < bin_start + 2:
                            rt_counts[bin_start] += 1
                            break

                # Convert data into the required format
                rt_data[file_name] = {bin_start: count for bin_start, count in rt_counts.items()}

    return rt_data

# Load the peptide counts data
rt_plot_data = load_peptide_counts_by_rt(tsv_folder_path)

# Generate line plots for each dataset
if rt_plot_data:
    line_pconfig = {
        'id': 'peptides_over_rt',
        'title': 'Number of Peptides over Retention Time',
        'xlab': 'Retention Time (minutes)',
        'ylab': 'Number of Peptides',
        'cpswitch': True,  # Enables clickable plot options
        'cpswitch_c_active': False,  # Keeps the plots in separate tabs
        'cpswitch_counts_label': 'Peptides Count by Retention Time',
        'cpswitch_percent_label': 'Peptides Percentage by Retention Time',
    }

    # Create the line plot
    module.add_section(
        plot=linegraph.plot(rt_plot_data, pconfig=line_pconfig),
        name="Peptides over Retention Time",
        anchor="peptides_over_rt_section",
    )
else:
    print("No data available for peptide counts over retention time.")


# Load and Plot Xcorr Distribution

In [45]:
# Function to load Xcorr data from TSV files
def load_xcorr(folder_path):
    xcorr_dict = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tsv"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as f:
                reader = csv.reader(f, delimiter='\t')
                headers = next(reader)
                rows = list(reader)
                if 'COMET:xcorr' not in headers:
                    log.warning(f"WARNING: 'COMET:xcorr' column not found in {file_name}")
                    continue
                xcorr_index = headers.index('COMET:xcorr')
                xcorr_values = [float(row[xcorr_index]) for row in rows if row[xcorr_index]]
                xcorr_dict[file_name] = xcorr_values
    return xcorr_dict

# Load Xcorr data
sample_xcorr_dict = load_xcorr(tsv_folder_path)

# Function to determine a common range and create bins
def create_common_bins(xcorr_dict, bin_size=0.1):
    # Find the overall min and max values across all datasets
    all_xcorr_values = [val for sublist in xcorr_dict.values() for val in sublist]
    min_xcorr = min(all_xcorr_values)
    max_xcorr = max(all_xcorr_values)

    # Define common bins across all datasets
    bins = [min_xcorr + i * bin_size for i in range(int((max_xcorr - min_xcorr) / bin_size) + 1)]
    return bins

# Function to bin Xcorr values for plotting and calculate frequencies
def bin_xcorr_values(xcorr_values, bins):
    if not xcorr_values:
        print("No Xcorr values to bin.")
        return {}
    
    # Initialize bin counts
    bin_counts = OrderedDict((f"{round(b, 2)}-{round(b + (bins[1] - bins[0]), 2)}", 0) for b in bins[:-1])
    
    # Count frequencies in bins
    for xcorr in xcorr_values:
        for i, b in enumerate(bins[:-1]):
            if b <= xcorr < bins[i + 1]:  # Ensures no overlap and proper bin assignment
                bin_counts[f"{round(b, 2)}-{round(b + (bins[1] - bins[0]), 2)}"] += 1
                break

    # Convert counts to frequencies
    total_count = sum(bin_counts.values())
    bin_frequencies = {k: (v / total_count) * 100 for k, v in bin_counts.items()}  # Convert to percentage
    return bin_frequencies

# Create common bins across all datasets
common_bins = create_common_bins(sample_xcorr_dict)

# Create frequency data for each sample's Xcorr values
xcorr_bin_frequencies = {sample: bin_xcorr_values(xcorr_values, common_bins) for sample, xcorr_values in sample_xcorr_dict.items()}

# Prepare data for the bar plot and ensure it's sorted by numeric range
xcorr_barplot_data = OrderedDict()
for sample, bin_frequencies in xcorr_bin_frequencies.items():
    for bin_range, frequency in bin_frequencies.items():
        if bin_range not in xcorr_barplot_data:
            xcorr_barplot_data[bin_range] = {}
        xcorr_barplot_data[bin_range][sample] = frequency

# Sort xcorr_barplot_data by the numeric start of each bin range
xcorr_barplot_data = OrderedDict(sorted(xcorr_barplot_data.items(), key=lambda x: float(x[0].split('-')[0])))

# Configuration for the Xcorr distribution bar plot
xcorr_pconfig = {
    'id': 'xcorr_distribution',
    'title': 'Xcorr Distribution (Frequency) for All Samples',
    'xlab': 'Xcorr Range',
    'ylab': 'Frequency [%]',
    'xmax': max(float(bin_range.split('-')[1]) for bin_range in xcorr_barplot_data),  # Dynamically set xmax
    'xmin': min(float(bin_range.split('-')[0]) for bin_range in xcorr_barplot_data),  # Dynamically set xmin
    'stacked': False
}

# Add the Xcorr distribution bar plot section
if xcorr_barplot_data:  # Ensure there is data to plot
    module.add_section(
        plot=bargraph.plot(xcorr_barplot_data, pconfig=xcorr_pconfig),
        name="Xcorr Distribution (Frequency)",
        anchor="xcorr_distribution_section",
    )
else:
    print("No data available for Xcorr distribution plot.")


# Generate and Display the MultiQC Report

In [46]:
# Ensure the custom data module is included in the report
multiqc.config.module_order = ['custom_data']
multiqc.config.report_title = "MHCquant QC Report"

# Add the module to the MultiQC report only once
multiqc.report.modules.append(module)

# Write the updated MultiQC report
multiqc.write_report(
    force=True,
    title="MHCquant QC Report",
    filename=os.path.join(report_path, 'mhcquant_multiqc_report.html')
)

# Display the report within Jupyter
display(IFrame(src=os.path.join(report_path, 'mhcquant_multiqc_report.html'), width='100%', height=1000))

2024-09-20 12:27:00,031 - multiqc.core.tmp_dir - DEBUG - Using temporary directory: /tmp/tmpgbegvxij
2024-09-20 12:27:00,032 - multiqc.core.update_config - DEBUG - This is MultiQC v1.23
2024-09-20 12:27:00,034 - multiqc.core.update_config - DEBUG - Running Python 3.9.19 (main, May  6 2024, 19:43:03)  [GCC 11.2.0]
2024-09-20 12:27:00,044 - multiqc.config - INFO - Loading config settings from: multiqc_config.yaml
            config | Loading config settings from: multiqc_config.yaml
2024-09-20 12:27:00,046 - multiqc.config - DEBUG - New config: {'log_filesize_limit': 2000000000, 'filesearch_lines_limit': 2000000000, 'custom_data': {'my_data_type': {'id': 'my_bar_plot', 'section_name': 'Custom Bar Plot', 'description': 'This section contains a custom bar plot with grouped colors.', 'plot_type': 'bargraph', 'pconfig': {'id': 'barplot_config', 'title': 'Custom Bar Plot', 'ylab': 'Mean Values', 'plot_groups': {'psm_file': {'name': 'psm_file', 'color': '#1f77b4'}, 'ms2pip': {'name': 'ms2pip',

/// https://multiqc.info 🔍 v1.23


2024-09-20 12:27:00,572 - multiqc.core.version_check - WARNING - MultiQC Version v1.25 now available!
     version_check | MultiQC Version v1.25 now available!
2024-09-20 12:27:00,578 - multiqc.core.version_check - DEBUG - Latest MultiQC version is v1.25, released 2024-09-17
2024-09-20 12:27:00,597 - multiqc.core.write_results - DEBUG - Rendering plots. This may take a while...


  rendering plots | █████████████████████████                 62% 5/8  Retention Time Correlation (DN02_Brain_1_mqc.tsv)


KeyboardInterrupt: 